In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://calendars.students.yorku.ca/2021-2022/programs/LE/mechanical-engineering"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

# 1. Locate tab where the courses are listed and collect the course links

In [4]:
driver.get(url)

In [5]:
driver.find_element_by_link_text("Degree Requirements").click()

In [7]:
driver.find_element_by_link_text("Program-Specific Degree Requirements").click()

In [8]:
page_soup = soup(driver.page_source, 'lxml')

In [9]:
containers = page_soup.find("div", "program-specific-degree-requirements-display").find("ul").findAll("a")
len(containers)

34

In [10]:
link_texts = [container.text.strip() for container in containers if container.text != "Complementary Studies" and container.text != "engineering program core"]

In [12]:
link_texts

['SC/MATH 2271 3.00',
 'LE/MECH 2112 3.00',
 'LE/MECH 2201 3.00',
 'LE/MECH 2202 3.00',
 'LE/MECH 2301 3.00',
 'LE/MECH 2302 3.00',
 'LE/MECH 2401 3.00',
 'LE/MECH 2412 3.00',
 'LE/MECH 2502 3.00',
 'LE/EECS 3505 3.00',
 'LE/MECH 3201 3.00',
 'LE/MECH 3202 3.00',
 'LE/MECH 3203 3.00',
 'LE/MECH 3302 3.00',
 'LE/MECH 3401 3.00',
 'LE/MECH 3409 3.00',
 'LE/MECH 3502 3.00',
 'LE/MECH 3503 3.00',
 'LE/MECH 3504 3.00',
 'LE/ENG 4550 3.00',
 'LE/MECH 4401 3.00',
 'LE/MECH 4402 4.00',
 'LE/MECH 4502 3.00',
 'LE/MECH 4504 3.00',
 'LE/MECH 4201 3.00',
 'LE/MECH 4202 3.00',
 'LE/MECH 4203 3.00',
 'LE/MECH 4301 3.00',
 'LE/MECH 4510 3.00',
 'LE/MECH 4511 3.00',
 'LE/MECH 4512 3.00',
 'LE/ENG 4650 3.00']

# 2. Implement automation script to scrape all courses

In [14]:
from selenium.common.exceptions import NoSuchElementException

course_codes = []
course_names = []
course_descs = []
counter = 0

driver.get(url)
time.sleep(3)
    
for link_text in link_texts:
    
    driver.find_element_by_link_text("Degree Requirements").click()
    time.sleep(1)
    driver.find_element_by_link_text("Program-Specific Degree Requirements").click()
    time.sleep(1)

    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue
    time.sleep(1)
    link.click()    
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    try:
        course_title = page_soup.find("h1").text.strip()
    except AttributeError:
        print("Course info unavailable")
        print()
        driver.back()
        time.sleep(3)
        continue
    course_code = re.split("   [0-9\.]+\s+", course_title)[0].strip()
    course_name = re.split("   [0-9\.]+\s+", course_title)[1].strip()
    course_desc = page_soup.find("p", text="Course Description:").findNextSibling().text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], " ", course_names[-1], "\n", course_descs[-1])
    print()
    counter += 1
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  SC/MATH 2271   Differential Equations for Scientists and Engineers 
 Introduction to ordinary and partial differential equations, including their classification, boundary conditions, and methods of solution. Equations, methods, and solutions relevant to science and engineering are emphasized, and exploration is encouraged with the aid of software. Three lecture hours per week. One term. Three credits. Prerequisites: One of SC/MATH 2015 3.00, SC/MATH 2310 3.00 or equivalent; one of SC/MATH 1025 3.00, SC/MATH 2022 3.00, SC/MATH 2222 3.00 or equivalent. Course Credit Exclusions: SC/MATH 2270 3.00, GL/MATH 3400 3.00.

Scraped  LE/MECH 2112   Mechanical Engineering: Professionalism and Society 
 In this course, students explore their role in the profession of mechanical engineering and the relationship between mechanical engineering and society. Students learn about the role of engineering in society, human-centred design principles, working with communities, and technological stew

Scraped  LE/MECH 3409   Machine Elements Design 
 Introduces methodology for mechanical design of components. It discusses topics including design for static and dynamic loads, failure analysis. fatigue, component design and selection for materials and machine elements, e.g. threaded joints, springs, gears, belt, chain, bearings, etc. Prerequisites: LE/MECH 2301 3.00, LE/MECH 2401 3.00. Course Credit Exclusion: LE/MECH 2409 3.00

Scraped  LE/MECH 3502   Solid Mechanics and Materials Laboratory 
 Building on the foundational knowledge in the mechanics of materials, this course introduces students to a number of measurement and characterization methods used for macro- and micro-systems. A selected number of laboratory experiments and demonstrations may include: strain measurements 
(e.g. strain gauges and/or speckles & interferometry method), deflection measurements, hardness, impact, non-destructive testing method for crack detection; material characterization methods including techniqu

Scraped  LE/MECH 4512   Principles of Bioengineering 
 Introduction to contemporary themes in bioengineering including: biological concepts for engineers; cell and tissue engineering; regenerative medicine and stem cells, bionanotechnology, biomaterials, drug screening, bioreactors, biotechnology, bioinformatics, genetic engineering, clinical trials and regulations. Pre-requisites: 21 3000-level science or engineering credits including MECH 3503 3.00
Date submitted: 2018-01-09

Scraped  LE/ENG  4650   Feedback Control Systems 
 This course teaches fundamentals of control design and analysis using state-space methods. This includes both the practical and theoretical aspects of the topic. The students are expected to design controllers using state-space methods and evaluate the control performance and validate if these controllers are robust to system uncertainties and external disturbances. Prerequisites: LE/ENG 4550 3.00 or the following combination of courses: SC/MATH 3410 3.00; SC/MA

# 3. Inspect and write to CSV

In [15]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,SC/MATH 2271,Differential Equations for Scientists and Engi...,Introduction to ordinary and partial different...
1,LE/MECH 2112,Mechanical Engineering: Professionalism and So...,"In this course, students explore their role in..."
2,LE/MECH 2201,Thermodynamics,This course covers properties and behaviour of...
3,LE/MECH 2202,Heat and Flow Engineering Principles,This course covers introduction to modes of he...
4,LE/MECH 2301,Mechanics of Material 1,This course covers normal and shear stresses a...
5,LE/MECH 2302,Dynamics,This course covers kinematics and kinetics of ...
6,LE/MECH 2401,Engineering Graphics & CAD Modeling,This course discusses technical drawing princi...
7,LE/MECH 2412,Mini Design Project 1,The course consists of two main modules. The f...
8,LE/MECH 2502,Instrumentation and Measurement Techniques,This course covers underlying physics and desi...
9,LE/EECS 3505,Electrical Systems for Mechanical Engineers,Many mechanical systems today are integrated w...


In [16]:
df.to_csv('York_MechEng_Core_and_Electives_Courses.csv', index = False)

In [17]:
driver.quit()